# 태스크 2b: 추상 텍스트 요약

이 노트북에서는 입력 텍스트가 모델 컨텍스트 길이를 초과하거나 환각 출력을 생성하거나 메모리 부족 오류를 트리거할 수 있는 큰 문서 요약에서 발생하는 문제를 관리할 수 있습니다.

이러한 문제를 완화하기 위해 이 노트북에서는 언어 모델을 활용하는 애플리케이션을 활성화하는 도구 키트인 [LangChain](https://python.langchain.com/docs/get_started/introduction.html) 프레임워크와 함께 프롬프트 청킹 및 체이닝을 사용하는 아키텍처를 보여줍니다.

사용자 문서가 토큰 한도를 초과할 때 발생하는 시나리오를 해결하는 접근 방식을 살펴봅니다. 청킹에서는 문서를 모델에 순차적으로 공급하기 전에 컨텍스트 길이 임곗값 미만의 세그먼트로 분할합니다. 이 체인에서 이전 컨텍스트를 유지하면서 청크 전체에 걸쳐 프롬프트를 표시합니다. 이 접근 방식을 적용하여 통화 대본, 회의 대본, 도서, 기사, 블로그 게시물 및 기타 관련 콘텐츠를 요약할 수 있습니다.

## 태스크 2b.1: 환경 설정

이 태스크에서는 환경을 설정합니다.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## 태스크 2b.2: 긴 텍스트 요약 

### Boto3로 LangChain 구성

이 태스크에서는 LangChain Bedrock 클래스용 LLM을 지정해야 하며, 추론을 위해 인수를 전달할 수 있습니다.

In [ ]:
# model configuration
from langchain_aws import BedrockLLM
modelId = "amazon.titan-text-premier-v1:0"
llm = BedrockLLM(
    model_id=modelId,
    model_kwargs={
        "maxTokenCount": 2048,
        "temperature": 0,
        "topP": 1
    },
    client=bedrock_client
)

## 태스크 2b.3: 많은 토큰을 포함하는 텍스트 파일 로드

이 태스크에서는 편지 디렉터리에서 [2022년 주주에게 보내는 Amazon의 CEO 편지](https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-2022-letter-to-shareholders)의 텍스트 파일을 찾을 수 있습니다. 다음 셀은 텍스트 파일을 로드하고 토큰 수를 계산합니다. 텍스트 파일의 토큰 수가 이 모델의 최대 토큰 수를 초과한다는 것을 나타내는 경고가 표시됩니다.

In [ ]:
#get tokens
shareholder_letter = "../letters/2022-letter.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()
    
llm.get_num_tokens(letter)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 경고를 무시하고 다음 셀로 넘어가도 됩니다.

## 태스크 2b.4: 긴 텍스트를 여러 청크로 분할

이 태스크에서는 텍스트가 너무 길어서 프롬프트에 맞지 않기 때문에 텍스트를 더 작은 청크로 분할합니다. LangChain의 `RecursiveCharacterTextSplitter`에서는 각 청크의 크기가 `chunk_size`보다 작아질 때까지 재귀적으로 긴 텍스트를 청크로 분할하는 것을 지원합니다. 텍스트는 `separators=["\n\n", "\n"]`를 사용해서 청크로 구분되므로 각 단락이 여러 청크로 분할되는 것을 방지할 수 있습니다.

청크당 6,000자를 사용하여 각 부분에 대한 요약을 개별적으로 얻을 수 있습니다. 한 청크 내의 토큰 수 또는 단어 조각 수는 텍스트에 따라 달라집니다.

In [ ]:
#chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [ ]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

## 태스크 2b.5: 청크 요약 및 조합

이 태스크에서는 토큰 수가 다른 문서에서도 일관될 것으로 가정하므로 순조롭게 진행할 수 있을 것입니다. LangChain의 `load_summarize_chain`을 사용하여 텍스트를 요약할 수 있습니다. `load_summarize_chain`에서는 세 가지 요약 방법인 `stuff`, `map_reduce` 및 `refine`을 제공합니다.

- `stuff`: 모든 청크를 하나의 프롬프트에 배치합니다. 따라서 최대 토큰 수 제한에 도달할 것입니다.
- `map_reduce`: 각 청크를 요약하고, 요약을 조합하고, 조합된 요약을 요약합니다. 조합한 요약이 너무 크면 오류가 발생합니다.
- `refine`: 첫 번째 청크를 요약한 다음, 첫 번째 요약으로 두 번째 청크를 요약합니다. 모든 청크가 요약될 때까지 동일한 프로세스가 반복됩니다.

map_reduce 및 refine은 둘 다 LLM을 여러 번 호출하고, 최종 요약을 얻을 때까지 기다립니다. 여기에서는 map_reduce를 시도해 볼 수 있습니다.

In [ ]:
# Set verbose=True if you want to see the prompts being used
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=False)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 문서 수, Bedrock 요청 속도 할당량 및 구성된 재시도 설정에 따라, 아래 체인을 실행하는 데 다소 시간이 소요될 수 있습니다.

In [ ]:
#invoke chain
output = ""
try:
    
    output = summary_chain.invoke(docs)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

In [ ]:
# print output
print(output['output_text'])

지금까지 LangChain 프레임워크를 통해 프롬프트 청킹 및 체이닝을 사용하여 긴 입력 텍스트에서 발생하는 문제를 완화하면서 큰 문서를 요약하는 실험을 해보았습니다.

### 직접 해보기
- 프롬프트를 특정 사용 사례로 변경하고 다양한 모델의 출력을 평가하십시오.
- 토큰 길이를 조작하면서 서비스의 지연 시간 및 응답성을 이해해 보십시오.
- 다양한 프롬프트 엔지니어링 원칙을 적용하여 더 나은 출력을 얻을 수 있습니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫고 **태스크 3**를 계속합니다.